In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels)) # real loss
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a') # max
#     ###################################################### ~Q(s,~a)= r # min
#     ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
#     targetQs = rewards + (gamma * aQs2_logits)
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # minimize Q
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # maximize Q
#     eloss += tf.reduce_mean(targetQs) # minimize Q
#     aloss2 += -tf.reduce_mean(targetQs) # maxizmie Q
#     eloss += tf.reduce_mean(eQs_logits) # minimize Q
#     aloss2 += -tf.reduce_mean(eQs_logits) # maxizmie Q
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:19.0000 R:19.0000 rate:0.0380 aloss:0.6929 eloss:1.4571 aloss2:1.3859 exploreP:0.9981
Episode:1 meanR:18.5000 R:18.0000 rate:0.0360 aloss:0.6922 eloss:1.4416 aloss2:1.3916 exploreP:0.9963
Episode:2 meanR:31.0000 R:56.0000 rate:0.1120 aloss:0.6921 eloss:1.3975 aloss2:1.4020 exploreP:0.9908
Episode:3 meanR:29.2500 R:24.0000 rate:0.0480 aloss:0.6903 eloss:1.3738 aloss2:1.4150 exploreP:0.9885
Episode:4 meanR:29.8000 R:32.0000 rate:0.0640 aloss:0.6885 eloss:1.3750 aloss2:1.4111 exploreP:0.9854
Episode:5 meanR:28.1667 R:20.0000 rate:0.0400 aloss:0.6868 eloss:1.3391 aloss2:1.4268 exploreP:0.9834
Episode:6 meanR:26.2857 R:15.0000 rate:0.0300 aloss:0.6887 eloss:1.3061 aloss2:1.4471 exploreP:0.9820
Episode:7 meanR:24.8750 R:15.0000 rate:0.0300 aloss:0.6884 eloss:1.3285 aloss2:1.4333 exploreP:0.9805
Episode:8 meanR:29.0000 R:62.0000 rate:0.1240 aloss:0.6867 eloss:1.2808 aloss2:1.4635 exploreP:0.9745
Episode:9 meanR:28.7000 R:26.0000 rate:0.0520 aloss:0.6864 eloss:1.2753 aloss2:1.4

Episode:80 meanR:25.8642 R:20.0000 rate:0.0400 aloss:0.7034 eloss:0.5832 aloss2:2.5748 exploreP:0.8129
Episode:81 meanR:26.3049 R:62.0000 rate:0.1240 aloss:0.6936 eloss:0.5505 aloss2:2.6124 exploreP:0.8079
Episode:82 meanR:26.1687 R:15.0000 rate:0.0300 aloss:0.6938 eloss:0.5519 aloss2:2.6511 exploreP:0.8067
Episode:83 meanR:26.0357 R:15.0000 rate:0.0300 aloss:0.6812 eloss:0.5473 aloss2:2.6821 exploreP:0.8055
Episode:84 meanR:25.8588 R:11.0000 rate:0.0220 aloss:0.6959 eloss:0.5544 aloss2:2.6256 exploreP:0.8047
Episode:85 meanR:25.8721 R:27.0000 rate:0.0540 aloss:0.6859 eloss:0.5590 aloss2:2.7220 exploreP:0.8025
Episode:86 meanR:25.7126 R:12.0000 rate:0.0240 aloss:0.6851 eloss:0.5685 aloss2:2.6587 exploreP:0.8016
Episode:87 meanR:25.8409 R:37.0000 rate:0.0740 aloss:0.6876 eloss:0.5469 aloss2:2.7287 exploreP:0.7986
Episode:88 meanR:25.7079 R:14.0000 rate:0.0280 aloss:0.6753 eloss:0.5431 aloss2:2.6912 exploreP:0.7975
Episode:89 meanR:25.6222 R:18.0000 rate:0.0360 aloss:0.6836 eloss:0.5342 

Episode:159 meanR:31.9500 R:26.0000 rate:0.0520 aloss:0.6789 eloss:0.4834 aloss2:3.5488 exploreP:0.6337
Episode:160 meanR:31.9200 R:13.0000 rate:0.0260 aloss:0.6685 eloss:0.4563 aloss2:3.5218 exploreP:0.6328
Episode:161 meanR:31.8200 R:21.0000 rate:0.0420 aloss:0.6810 eloss:0.4921 aloss2:3.5707 exploreP:0.6315
Episode:162 meanR:31.9500 R:40.0000 rate:0.0800 aloss:0.6729 eloss:0.5007 aloss2:3.5806 exploreP:0.6291
Episode:163 meanR:31.9500 R:57.0000 rate:0.1140 aloss:0.6765 eloss:0.5037 aloss2:3.5237 exploreP:0.6255
Episode:164 meanR:31.8900 R:23.0000 rate:0.0460 aloss:0.6754 eloss:0.4728 aloss2:3.5408 exploreP:0.6241
Episode:165 meanR:31.6400 R:32.0000 rate:0.0640 aloss:0.6785 eloss:0.5037 aloss2:3.5605 exploreP:0.6222
Episode:166 meanR:31.3700 R:11.0000 rate:0.0220 aloss:0.6768 eloss:0.4994 aloss2:3.5404 exploreP:0.6215
Episode:167 meanR:32.7100 R:165.0000 rate:0.3300 aloss:0.6763 eloss:0.4910 aloss2:3.5449 exploreP:0.6115
Episode:168 meanR:32.8600 R:51.0000 rate:0.1020 aloss:0.6758 el

Episode:238 meanR:55.9400 R:151.0000 rate:0.3020 aloss:0.6772 eloss:0.5051 aloss2:3.4876 exploreP:0.3954
Episode:239 meanR:56.6100 R:118.0000 rate:0.2360 aloss:0.6732 eloss:0.5184 aloss2:3.4638 exploreP:0.3908
Episode:240 meanR:57.0400 R:61.0000 rate:0.1220 aloss:0.6733 eloss:0.5290 aloss2:3.4560 exploreP:0.3885
Episode:241 meanR:58.3800 R:159.0000 rate:0.3180 aloss:0.6761 eloss:0.5026 aloss2:3.4673 exploreP:0.3826
Episode:242 meanR:58.4400 R:49.0000 rate:0.0980 aloss:0.6783 eloss:0.4950 aloss2:3.4856 exploreP:0.3807
Episode:243 meanR:59.1500 R:101.0000 rate:0.2020 aloss:0.6744 eloss:0.5293 aloss2:3.4840 exploreP:0.3770
Episode:244 meanR:59.4800 R:49.0000 rate:0.0980 aloss:0.6737 eloss:0.5156 aloss2:3.4383 exploreP:0.3752
Episode:245 meanR:59.7200 R:87.0000 rate:0.1740 aloss:0.6752 eloss:0.4998 aloss2:3.4673 exploreP:0.3721
Episode:246 meanR:59.7400 R:46.0000 rate:0.0920 aloss:0.6692 eloss:0.5410 aloss2:3.4988 exploreP:0.3704
Episode:247 meanR:60.3600 R:86.0000 rate:0.1720 aloss:0.6740

Episode:317 meanR:88.0800 R:132.0000 rate:0.2640 aloss:0.6990 eloss:0.5204 aloss2:3.3799 exploreP:0.1975
Episode:318 meanR:88.8700 R:143.0000 rate:0.2860 aloss:0.7089 eloss:0.5234 aloss2:3.3823 exploreP:0.1949
Episode:319 meanR:90.8200 R:277.0000 rate:0.5540 aloss:0.7187 eloss:0.5086 aloss2:3.3761 exploreP:0.1898
Episode:320 meanR:91.5100 R:158.0000 rate:0.3160 aloss:0.7138 eloss:0.5027 aloss2:3.3915 exploreP:0.1870
Episode:321 meanR:91.9800 R:130.0000 rate:0.2600 aloss:0.7205 eloss:0.4979 aloss2:3.3920 exploreP:0.1847
Episode:322 meanR:92.1200 R:98.0000 rate:0.1960 aloss:0.7125 eloss:0.5287 aloss2:3.4007 exploreP:0.1830
Episode:323 meanR:92.2900 R:98.0000 rate:0.1960 aloss:0.7292 eloss:0.5197 aloss2:3.3722 exploreP:0.1813
Episode:324 meanR:93.0600 R:130.0000 rate:0.2600 aloss:0.7015 eloss:0.5148 aloss2:3.3723 exploreP:0.1791
Episode:325 meanR:93.8000 R:88.0000 rate:0.1760 aloss:0.7006 eloss:0.5408 aloss2:3.3770 exploreP:0.1776
Episode:326 meanR:95.0700 R:187.0000 rate:0.3740 aloss:0.7

...

Episode:1470 meanR:208.3000 R:500.0000 rate:1.0000 aloss:0.3403 eloss:-10.7410 aloss2:56.1488 exploreP:0.0100
Episode:1471 meanR:207.0000 R:370.0000 rate:0.7400 aloss:0.3363 eloss:-22.1203 aloss2:74.2376 exploreP:0.0100
Episode:1472 meanR:207.0000 R:500.0000 rate:1.0000 aloss:0.3264 eloss:-12.0003 aloss2:58.2370 exploreP:0.0100
Episode:1473 meanR:211.9000 R:500.0000 rate:1.0000 aloss:0.3320 eloss:-14.5334 aloss2:62.5661 exploreP:0.0100
Episode:1474 meanR:211.9000 R:500.0000 rate:1.0000 aloss:0.3315 eloss:-10.7978 aloss2:56.8150 exploreP:0.0100
Episode:1475 meanR:216.5400 R:500.0000 rate:1.0000 aloss:0.3289 eloss:-16.4341 aloss2:66.1561 exploreP:0.0100
Episode:1476 meanR:221.4000 R:500.0000 rate:1.0000 aloss:0.3238 eloss:-11.7356 aloss2:58.6968 exploreP:0.0100
Episode:1477 meanR:221.4000 R:10.0000 rate:0.0200 aloss:0.3134 eloss:-11.5071 aloss2:58.1961 exploreP:0.0100
Episode:1478 meanR:221.4000 R:10.0000 rate:0.0200 aloss:0.3251 eloss:-11.8168 aloss2:58.9295 exploreP:0.0100
Episode:1479

Episode:1545 meanR:224.4700 R:50.0000 rate:0.1000 aloss:0.3278 eloss:-10.9788 aloss2:65.0776 exploreP:0.0100
Episode:1546 meanR:219.5700 R:10.0000 rate:0.0200 aloss:0.3307 eloss:-9.3712 aloss2:62.9433 exploreP:0.0100
Episode:1547 meanR:218.8900 R:182.0000 rate:0.3640 aloss:0.3302 eloss:-12.6734 aloss2:68.1033 exploreP:0.0100
Episode:1548 meanR:217.8800 R:103.0000 rate:0.2060 aloss:0.3283 eloss:-13.9753 aloss2:70.6167 exploreP:0.0100
Episode:1549 meanR:220.2800 R:250.0000 rate:0.5000 aloss:0.3286 eloss:-10.7900 aloss2:64.9311 exploreP:0.0100
Episode:1550 meanR:215.3800 R:10.0000 rate:0.0200 aloss:0.3304 eloss:-9.6521 aloss2:63.2207 exploreP:0.0100
Episode:1551 meanR:213.5900 R:11.0000 rate:0.0220 aloss:0.3319 eloss:-11.1183 aloss2:65.6544 exploreP:0.0100
Episode:1552 meanR:213.5900 R:10.0000 rate:0.0200 aloss:0.3292 eloss:-10.6799 aloss2:65.7582 exploreP:0.0100
Episode:1553 meanR:209.1700 R:58.0000 rate:0.1160 aloss:0.3300 eloss:-12.3504 aloss2:67.5170 exploreP:0.0100
Episode:1554 meanR

Episode:1620 meanR:171.7800 R:12.0000 rate:0.0240 aloss:0.3124 eloss:-21.9000 aloss2:84.0490 exploreP:0.0100
Episode:1621 meanR:171.0300 R:10.0000 rate:0.0200 aloss:0.3196 eloss:-39.5928 aloss2:116.7245 exploreP:0.0100
Episode:1622 meanR:166.1300 R:10.0000 rate:0.0200 aloss:0.3028 eloss:-19.1484 aloss2:77.9637 exploreP:0.0100
Episode:1623 meanR:166.1300 R:500.0000 rate:1.0000 aloss:0.3018 eloss:-13.9937 aloss2:68.9628 exploreP:0.0100
Episode:1624 meanR:166.1300 R:500.0000 rate:1.0000 aloss:0.2983 eloss:-17.0356 aloss2:74.7156 exploreP:0.0100
Episode:1625 meanR:166.1300 R:10.0000 rate:0.0200 aloss:0.3013 eloss:-13.9265 aloss2:69.4225 exploreP:0.0100
Episode:1626 meanR:166.9400 R:91.0000 rate:0.1820 aloss:0.2964 eloss:-15.4294 aloss2:71.6695 exploreP:0.0100
Episode:1627 meanR:162.2200 R:28.0000 rate:0.0560 aloss:0.2931 eloss:-14.9917 aloss2:70.4472 exploreP:0.0100
Episode:1628 meanR:159.7600 R:10.0000 rate:0.0200 aloss:0.2922 eloss:-14.1282 aloss2:69.0355 exploreP:0.0100
Episode:1629 mea

Episode:1695 meanR:181.9500 R:10.0000 rate:0.0200 aloss:0.3105 eloss:-10.5225 aloss2:70.5953 exploreP:0.0100
Episode:1696 meanR:181.9500 R:10.0000 rate:0.0200 aloss:0.3210 eloss:-10.3175 aloss2:69.9902 exploreP:0.0100
Episode:1697 meanR:181.9600 R:11.0000 rate:0.0220 aloss:0.3096 eloss:-10.7084 aloss2:71.0803 exploreP:0.0100
Episode:1698 meanR:186.6800 R:482.0000 rate:0.9640 aloss:0.3205 eloss:-10.3978 aloss2:70.9980 exploreP:0.0100
Episode:1699 meanR:181.7900 R:11.0000 rate:0.0220 aloss:0.3167 eloss:-8.3704 aloss2:68.3786 exploreP:0.0100
Episode:1700 meanR:185.0500 R:500.0000 rate:1.0000 aloss:0.3175 eloss:-9.4006 aloss2:70.4943 exploreP:0.0100
Episode:1701 meanR:185.0000 R:10.0000 rate:0.0200 aloss:0.3211 eloss:-7.9024 aloss2:67.6439 exploreP:0.0100
Episode:1702 meanR:185.0000 R:10.0000 rate:0.0200 aloss:0.3193 eloss:-8.7311 aloss2:69.9195 exploreP:0.0100
Episode:1703 meanR:189.9000 R:500.0000 rate:1.0000 aloss:0.3162 eloss:-9.6269 aloss2:70.8931 exploreP:0.0100
Episode:1704 meanR:18

Episode:1770 meanR:186.6100 R:10.0000 rate:0.0200 aloss:0.2952 eloss:-18.9304 aloss2:94.8819 exploreP:0.0100
Episode:1771 meanR:191.2300 R:500.0000 rate:1.0000 aloss:0.2999 eloss:-21.1791 aloss2:99.4550 exploreP:0.0100
Episode:1772 meanR:195.7500 R:462.0000 rate:0.9240 aloss:0.2987 eloss:-19.3404 aloss2:96.6162 exploreP:0.0100
Episode:1773 meanR:200.6400 R:500.0000 rate:1.0000 aloss:0.2948 eloss:-20.4891 aloss2:98.1843 exploreP:0.0100
Episode:1774 meanR:195.9700 R:33.0000 rate:0.0660 aloss:0.3005 eloss:-19.1231 aloss2:96.0825 exploreP:0.0100
Episode:1775 meanR:200.8700 R:500.0000 rate:1.0000 aloss:0.2948 eloss:-19.7398 aloss2:96.7858 exploreP:0.0100
Episode:1776 meanR:203.4300 R:432.0000 rate:0.8640 aloss:0.2949 eloss:-20.9515 aloss2:98.8578 exploreP:0.0100
Episode:1777 meanR:200.8500 R:10.0000 rate:0.0200 aloss:0.2903 eloss:-19.9334 aloss2:97.1477 exploreP:0.0100
Episode:1778 meanR:201.9500 R:120.0000 rate:0.2400 aloss:0.2918 eloss:-20.4636 aloss2:97.4064 exploreP:0.0100
Episode:1779 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
